In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
from tqdm import tqdm, tqdm_notebook

# Local imports
from preprocessing import clean_tweets, one_hot_encode, make_debug_df

### First load and clean the hatebase terms

In [ ]:
# Clean the hatebase slurs
def clean_slurs(text):
    """Lowercase and underscore join slur words."""
    return text.strip().lower().replace(' ', '_')

slurs = pd.read_csv('../data/original_hatebase_slurs.txt', header=None)

# Clean slurs
slur_list = [*map(lambda s: s.lower(), slurs[0].values)]
cleaned_slurs = [*map(clean_slurs, slur_list)]
pluralize_slurs = [s + end for s in cleaned_slurs for end in ['s', 'es']]
full_slur_list = sorted(pluralize_slurs + cleaned_slurs)

# Outputs
slur_map = {s: cs for s, cs in zip(slur_list, cleaned_slurs) if s != cs}
#out_slurs = pd.DataFrame(full_slur_list)
#out_slurs.to_csv('data/hatebase_slurs.txt', index=None, header=None, encoding='utf-8')

In [ ]:
def clean_slurs_in_context(text):
    """Replace slurs with their cleaned versions."""
    for k, v in slur_map.items():
        text = text.replace(k, v)
    return text

def extract_slurs(text):
    """Get a list of all slurs used in the text."""
    text = text.split(' ')
    all_slurs = []
    for s in full_slur_list:
        if s in text:
            all_slurs += [s]
    return all_slurs

### Davidson et al data

In [ ]:
path = '../data/davidson/'
fname = '{}labeled_data.csv'.format(path)

In [ ]:
# Load the data
df = pd.read_csv(fname, encoding='utf-8', index_col='Unnamed: 0').sample(frac=1)
label_map = {0: 'hate_speech', 1: 'offensive_language', 2: 'neither'}

In [ ]:
# Clean the tweets
df = clean_tweets(df)

# Convert columns to one hot encoding
df[['hate_speech', 'offensive_language', 'neither']] = \
    one_hot_encode(df['class'])
df['label'] =df['class'].apply(lambda c: label_map[c])
    
# Clean hate speech terms, and extract slurs
df['tweet'] = df['tweet'].apply(clean_slurs_in_context)
df['slurs'] = df['tweet'].apply(extract_slurs)
    
# Re-order the DataFrame, and drop some columns
df = df[['tweet', 'label', 'mentions', 'hashtags', 'slurs', 'original_tweet',
         'hate_speech', 'offensive_language', 'neither']]

In [ ]:
# Make a test/dev/train split
train_perc = 0.80
msk = np.random.rand(len(df)) < train_perc
train = df[msk]
not_train = df[~msk]
half = int(len(not_train) / 2)
dev = not_train[:half]
test = not_train[half:]

train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [ ]:
debug = make_debug_df(df)

In [ ]:
debug.to_csv('{}debug.csv'.format(path), index=False, encoding='utf-8')
train.to_csv('{}train.csv'.format(path), index=False, encoding='utf-8')
dev.to_csv('{}dev.csv'.format(path), index=False, encoding='utf-8')
test.to_csv('{}test.csv'.format(path), index=False, encoding='utf-8')

### Zeerak data

In [ ]:
path = '../data/zeerak_naacl/'
fname = '{}zeerak_naacl_tweets.csv'.format(path)
# Load the data
df2 = pd.read_csv(fname, encoding='utf-8').sample(frac=1)

In [ ]:
# First mask out some missing data
msk = ~ df2['label'].apply(lambda t: type(t) is float)
df2 = df2[msk]

In [ ]:
# Clean the tweets
df2.rename(index=str, columns={'text': 'tweet'}, inplace=True)
df2 = clean_tweets(df2)

# Label cleanup to match the other df format
labels = ['racism', 'sexism', 'none']
one_hot_label = [labels.index(l) for l in df2['label']]
for l in labels:
    df2[l] = -1

# Convert columns to one hot encoding
df2[['racism', 'sexism', 'none']] = \
    one_hot_encode(one_hot_label)
    
# Clean hate speech terms, and extract slurs
df2['tweet'] = df2['tweet'].apply(clean_slurs_in_context)
df2['slurs'] = df2['tweet'].apply(extract_slurs)
    
# Re-order the DataFrame, and drop some columns
df2 = df2[['tweet', 'label', 'mentions', 'hashtags', 'slurs', 'original_tweet',
           'racism', 'sexism', 'none', 'tweet_id', 'user_screen_name']]

In [ ]:
# Make a test/dev/train split
train_perc = 0.80
msk = np.random.rand(len(df2)) < train_perc
train = df2[msk]
not_train = df2[~msk]
half = int(len(not_train) / 2)
dev = not_train[:half]
test = not_train[half:]

train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [ ]:
debug = make_debug_df(df2, cols=labels)

In [ ]:
debug.to_csv('{}debug.csv'.format(path), index=False, encoding='utf-8')
train.to_csv('{}train.csv'.format(path), index=False, encoding='utf-8')
dev.to_csv('{}dev.csv'.format(path), index=False, encoding='utf-8')
test.to_csv('{}test.csv'.format(path), index=False, encoding='utf-8')

### Wiki talk data

In [ ]:
path = '../data/wiki_talk/'
fname = '{}labeled_data.csv'.format(path)

In [ ]:
# Load the data
df3 = pd.read_csv(fname, encoding='utf-8')# , index_col='Unnamed: 0').sample(frac=1)

In [ ]:
i_to_label = {0: 'toxic', 1: 'severe_toxic', 2: 'obscene', 3: 'threat',
              4: 'insult', 5: 'identity_hate'}
def one_hot_to_label(row):
    """Convert a one-hot vector to labels."""
    l = 'none'
    for i, v in enumerate(row):
        if v == 1:
            l = i_to_label[i]
    return l

In [ ]:
# Clean the tweets
print('Cleaning...')
df3.rename(index=str, columns={'comment_text': 'tweet'}, inplace=True)
df3 = clean_tweets(df3)

# Label cleanup to match the other df format
print('Fixing Labels...')
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
df3['label'] = df3[labels].apply(one_hot_to_label, axis=1)
df3['none'] = [*map(int, df3['label'] == 'none')]
    
# Clean hate speech terms, and extract slurs
print('Extracting Slurs...')
df3['tweet'] = df3['tweet'].apply(clean_slurs_in_context)
df3['slurs'] = df3['tweet'].apply(extract_slurs)

# Re-order the DataFrame, and drop some columns
df3 = df3[['tweet', 'label', 'mentions', 'hashtags', 'slurs', 'original_tweet'] +
           labels + ['none', 'id']]

In [ ]:
# Make a test/dev/train split
train_perc = 0.80
msk = np.random.rand(len(df3)) < train_perc
train = df3[msk]
not_train = df3[~msk]
half = int(len(not_train) / 2)
dev = not_train[:half]
test = not_train[half:]

train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)
dev.reset_index(drop=True, inplace=True)

In [ ]:
train.to_csv('{}train.csv'.format(path), index=False, encoding='utf-8')
dev.to_csv('{}dev.csv'.format(path), index=False, encoding='utf-8')
test.to_csv('{}test.csv'.format(path), index=False, encoding='utf-8')

In [ ]:
# Print out some examples
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

neutral_msk = np.array([True]*len(df))
for l in labels:
    neutral_msk = neutral_msk & ~np.array([*map(bool, df[l])])
msk2 = [len(text) < 60 for text in df['comment_text']]

# Print total
print("Total comments: {}".format(len(df)))

# Print neutral
print("Neutral comments: {}".format(sum(neutral_msk)))
msk = [m1 and m2 for m1, m2 in zip(neutral_msk, msk2)]
for ex in df[msk]['comment_text'].sample(5):
    print('\t', ex.replace('\n', ' '))

for l in labels:
    print("{} comments: {}".format(l.title(), sum(df[l])))
    msk1 = [*map(bool, df[l])]
    msk = [m1 and m2 for m1, m2 in zip(msk1, msk2)]
    for ex in df[msk]['comment_text'].sample(5):
        print('\t', ex.replace('\n', ' '))

### Looking at number of hatebase terms in our data

In [ ]:
Counter([b for a in df3['slurs'] for b in a]).most_common()

In [ ]:
sum(Counter([b for a in df3['slurs'] for b in a]).values())

In [ ]:
Counter([b for a in df['slurs'] for b in a]).most_common()